In [82]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import pandas as pd
import pickle, csv
import numpy as np
import time
import random
from pprint import pprint
from gensim.models.keyedvectors import KeyedVectors
torch.manual_seed(3)

### 1. Bilstm +CRF Model

In [83]:


'''Auxiliary functions'''

#Load pretrained embeddings function 
def load_embeddings(path_to_embeddings, word2id, embedding_dim = 25):
    '''
    Input: This only takes in w2v format. So convert all other embedding types/vectors to w2v format: 
    Output: Torch variable with embeddings only belonging to and indexed by word2id
    
    For glove:
    Use python -m gensim.scripts.glove2word2vec -i <GloVe vector file> -o <Word2vec vector file>
    as per https://radimrehurek.com/gensim/scripts/glove2word2vec.html
    
    
    W2V format is:
    <Num vectors> <dimensionality>
    <word1> <vector rep>
    <word2> <vector rep>
    ....and so on
    '''
    word_vectors = KeyedVectors.load_word2vec_format(path_to_embeddings, binary = False)
    
    
    corpus_embeddings = np.zeros((len(word2id), embedding_dim))
    #Should words not found in embedding file be random or zeros? 
    #Currently 0 and then they are trained anyway
    '''Should we use a try/catch block here so that words that are not in vocab force an exception?
    Or should we choose every word in vocab and see if it is in embedding vocab-> worst case |V|*|embedding_vocab|
    Or vice versa, same worst case but early stopping 
    Or use key retrieval so all words in embedding vocab are checked if they belong to dictionary key
    '''
    for word in word2id.keys():
        if word in word_vectors.vocab:
            corpus_embeddings[word2id[word]] = np.array(word_vectors[word])
        
    return torch.from_numpy(corpus_embeddings).float()


'''Training the simple encoder-decoder'''
def form_indexed_sequence_variable(sequence, word_to_id, end_token):
    #Input sequence is a list of word tokens
    #Output sequence is a list of id tokens based on provided word to id mapping
    input_seq = [word_to_id[token] for token in sequence] + [end_token]
    input_seq_var = autograd.Variable(torch.LongTensor(input_seq)).view(-1,1) #make a 2 rank tensor
    if(use_cuda):
        return input_seq_var.cuda()
    else:
        return input_seq_var
    
def generate_training_sequence(training_dataset, word_to_id, end_token):
    '''Generator function to yield training example 
    Not very useful in this case but for large datasets it can save memory if we pass the dataset by reference.
    Use hpy (guppy for py2) to explore memory usage in the heap
    '''
    random.shuffle(training_dataset) 
    for training_example in training_dataset:
        yield(form_indexed_sequence_variable(training_example[0], word_to_id, end_token), form_indexed_sequence_variable(training_example[1], word_to_id, end_token))
        
    
def to_scalar(var):
    # returns a python float
    return var.view(-1).data.tolist()[0]


def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
    
'''Final model-> Have to make batchwise'''
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, with_pretrained_embeddings = False, pretrained_embedding_matrix = None, freeze_embeddings = False):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        if(not with_pretrained_embeddings):
            self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        else:
            assert vocab_size == pretrained_embedding_matrix.size()[0]
            assert embedding_dim == pretrained_embedding_matrix.size()[1]
            self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
            self.word_embeds.weight = nn.Parameter(pretrained_embedding_matrix)
            if(freeze_embeddings):
                self.word_embeds.requires_grad = False
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j. 
        '''Imp:
        Transition probs are log likelihoods (log(p(X))) therefore range is -infinite (log(0)) to 0 (log(1))
        '''
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[start_tag], :] = -10000 #
        self.transitions.data[:, tag_to_ix[end_tag]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)),
                autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[start_tag]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = autograd.Variable(init_alphas)

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward variables at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[end_tag]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = autograd.Variable(torch.Tensor([0]))
        tags = torch.cat([torch.LongTensor([self.tag_to_ix[start_tag]]), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[end_tag], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_vvars[0][self.tag_to_ix[start_tag]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = autograd.Variable(init_vvars)
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id])
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to end_tag
        terminal_var = forward_var + self.transitions[self.tag_to_ix[end_tag]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[start_tag]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        '''Cost function'''
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

### 2. Loading dataset 

In [62]:
#Load relevant data for training
def get_dataset_info(file_name = "laptop", training_data_additional = False, get_inverse = False, lower = True):
    
    if(lower):
        opt_string = "lower" #hardcoded
    else:
        opt_string = "final"
        
        
    if(training_data_additional):
        training_data = pickle.load(open("Final_data/{}_{}_additional_training_list.pickle".format(file_name, opt_string)))
    else:
        training_data = pickle.load(open("Final_data/{}_{}_normal_training_list.pickle".format(file_name, opt_string)))
        
    tag2id = pickle.load(open("Final_data/{}_{}_tag2id.pickle".format(file_name, opt_string)))
    
    vocab2id = pickle.load(open("Final_data/{}_{}_vocab.pickle".format(file_name, opt_string)))
    
    if("<START>" in vocab2id.keys()): #assuming <START> and <END> always co-occur
        start_token = vocab2id["<START"]
        end_token = vocab2id["<END>"]
    else:
        start_token = vocab2id["<START>"] = len(vocab2id)
        end_token = vocab2id["<END>"] = len(vocab2id)
    
    if(not get_inverse):
        return training_data, tag2id, vocab2id, start_token, end_token
    else:
        id2vocab = {id_:token for token, id_ in vocab2id.items()}
        id2tag = {id_:tag for tag, id_ in tag2id.items()}
        
        return training_data, tag2id, vocab2id, start_token, end_token, id2vocab, id2tag

In [63]:
laptop_data, tag2id, laptop_vocab2id, start_token, end_token = get_dataset_info("laptop", True)
vocab_len = len(laptop_vocab2id)
num_total_examples = len(laptop_data)


training_first_index = 0
training_last_index = int(num_total_examples*0.7)
testing_first_index = training_last_index + 1

dataset_aoext = map(lambda x: (x[0],x[1]) , laptop_data) 
training_data_aoext = dataset_aoext[: training_last_index+1]
testing_data_aoext = dataset_aoext[testing_first_index:]

id2tag = {id_: tag for tag, id_ in tag2id.items()}
assert len(training_data_aoext) + len(testing_data_aoext)  == len(dataset_aoext) #Sanity check

### 3. Initializing Model 

In [64]:
'''Hyperparameters'''

start_tag = "<START>"
end_tag = "<END>"

embed_dim = 50
hidden_dim = 10
pretrained_embedding_matrix = load_embeddings("./Final_data/embeddings/glove/glove50dimw2v_format.txt", laptop_vocab2id, 50)

model = BiLSTM_CRF(vocab_len, tag2id, embed_dim, hidden_dim, True, pretrained_embedding_matrix, True)
optimizer = optim.SGD(model.parameters(), lr = 0.01, weight_decay = 1e-4)

In [71]:
#Check predictions before training
precheck_sent = prepare_sequence(training_data_aoext[0][0], laptop_vocab2id)
precheck_tags = torch.LongTensor(training_data_aoext[0][1])
print("Loss and predictions: ", model(precheck_sent))
print("Correct sequence: ")
pprint(precheck_tags)


('Loss and predictions: ', (Variable containing:
 29.1685
[torch.FloatTensor of size 1]
, [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]))
Correct sequence: 

 5
 5
 5
 5
 5
 5
 5
 5
 5
 1
 5
 5
 5
 5
 5
 3
 1
 2
[torch.LongTensor of size 18]



### 4. Training the model

In [88]:
def generate_training_sequence(training_data, token_to_id):
    '''Generator function to yield training example 
    Not very useful in this case but for large datasets it can save memory if we pass the dataset by reference.
    Use hpy (guppy for py2) to explore memory usage in the heap
    '''
    random.shuffle(training_data)
    for training_example in training_data:
        yield (prepare_sequence(training_example[0], token_to_id), torch.LongTensor(training_example[1]) )
        
def generate_testing_sequence(testing_data, token_to_id, testing_size = 500):
    random.shuffle(testing_data)
    for testing_example in testing_data[:testing_size]:
        yield(prepare_sequence(testing_example[0], token_to_id), torch.LongTensor(testing_example[1]))


In [97]:
'''Training'''
def train(model, n_epochs, optimizer, token_to_id, training_data, testing_data, testing_size = 100):
    
    for epoch in range(n_epochs):
        print("At epoch: {}".format(epoch))
        t1 = time.time()
     
    
        training_gen = generate_training_sequence(training_data, token_to_id)
        loss = 0
        iters = 0
    
        for sentence_in, targets in training_gen:
            model.zero_grad() #r1) remove all accumulated gradients from previous batch
        
          
        
            #3) Run forward pass 
            neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets)
            loss += neg_log_likelihood
        
            if(iters%500==0):
                tm = time.time()
                print("At iteration {}, time taken: {}".format(iters, tm-t1))
            iters+=1
            
            #4) Compute gradients, run backprop based on optimizer
            neg_log_likelihood.backward()
            optimizer.step()
            
        print("Training Loss for epoch: {}".format(loss/iters))
        t2 = time.time()
        print("time for epoch: {}".format(t2-t1))
        
        
        te_loss = 0 
        te_iters = 0
        testing_gen = generate_testing_sequence(testing_data, token_to_id, testing_size)

        for sentence_in, targets in testing_gen:
            neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets)
            te_loss += neg_log_likelihood
            
        print("Testing Loss for epoch: {}".format(te_loss/iters))

In [ ]:
train(model, 40, optimizer, laptop_vocab2id, training_data_aoext, testing_data_aoext)

At epoch: 0
At iteration 0, time taken: 0.0226540565491
At iteration 500, time taken: 22.6349010468
At iteration 1000, time taken: 40.6797330379
At iteration 1500, time taken: 56.1470930576
At iteration 2000, time taken: 71.1635129452
Training Loss for epoch: Variable containing:
 2.6766
[torch.FloatTensor of size 1]

time for epoch: 74.8348219395
Testing Loss for epoch: Variable containing:
 0.1778
[torch.FloatTensor of size 1]

At epoch: 1
At iteration 0, time taken: 1.95178198814
At iteration 500, time taken: 17.6402630806
At iteration 1000, time taken: 32.1657540798
At iteration 1500, time taken: 47.6366760731
At iteration 2000, time taken: 62.4609210491
Training Loss for epoch: Variable containing:
 2.4356
[torch.FloatTensor of size 1]

time for epoch: 66.4103529453
Testing Loss for epoch: Variable containing:
 0.1406
[torch.FloatTensor of size 1]

At epoch: 2
At iteration 0, time taken: 1.96679496765
At iteration 500, time taken: 18.7126209736
At iteration 1000, time taken: 36.74

In [100]:
torch.save(model.state_dict(), "Trained_models/ABSA_models/glove50dims/Bilstm_CRF_state.pth")
#torch.save(model, "Trained_models/ABSA_models/glove50dims/Bilstm_CRF.pth")